### 1. Library Import 

In [4]:
import re
from IPython.display import display
import pandas as pd

### 2. Data Import

In [6]:
df=pd.read_csv("fichier_res_cairn.csv", sep=";")
display(df) #88962 rows

,Unnamed: 0,url,titre,auteur,section,twitter_card,twitter_title,citation_titre,citation_auteur,journal,...,numero,page_debut,page_fin,doi,pdf_url,resume,texte_article,type_document,variante_recherchee,name
0,0,https://shs.cairn.info/orients-occidents-25-si...,Chapitre 11. « Grattez le Russe et vous trouve...,Thierry Camous,Histoire,summary,Chapitre 11. « Grattez le Russe et vous trouve...,Chapitre 11. « Grattez le Russe et vous trouve...,Thierry Camous,Hors collection,...,NaN,245,284,NaN,NaN,NaN,"C et aphorisme français, remontant sans doute ...",Chapitre d’ouvrage,Catherine 2 de Russie,Catherine II de Russie
1,1,https://shs.cairn.info/revoltes-et-revolutions...,Chapitre 14. La Russie face au servage et à la...,Robert Calvet,NaN,summary,Chapitre 14. La Russie face au servage et à la...,Chapitre 14. La Russie face au servage et à la...,Robert Calvet,Collection U,...,NaN,206,216,NaN,NaN,NaN,L e sujet proposé au concours s’ouvre avec une...,Chapitre d’ouvrage,Catherine 2 de Russie,Catherine II de Russie
2,2,https://shs.cairn.info/catherine-ii--978226206...,Bibliographie,NaN,NaN,summary,Bibliographie,Bibliographie,NaN,Hors collection,...,NaN,441,457,10.3917/perri.liech.2021.01.0441,NaN,NaN,NaN,Chapitre d’ouvrage,Catherine 2 de Russie,Catherine II de Russie
3,3,https://shs.cairn.info/l-europe-au-19-e-siecle...,Bibliographie,NaN,NaN,summary,Bibliographie,Bibliographie,NaN,Collection U,...,NaN,455,483,10.3917/arco.caron.2019.01.0455,NaN,NaN,NaN,Chapitre d’ouvrage,Catherine 2 de Russie,Catherine II de Russie
4,4,https://shs.cairn.info/histoire-de-la-russie-e...,11 - La technique du pouvoir,Michel Heller,Science politique,summary,11 - La technique du pouvoir,11 - La technique du pouvoir,Michel Heller,Tempus,...,NaN,810,821,NaN,NaN,NaN,C atherine II est nettement plus cultivée que ...,Chapitre d’ouvrage,Catherine 2 de Russie,Catherine II de Russie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88957,88957,https://shs.cairn.info/le-defi-des-emergents--...,L’Ukraine après Maïdan,Anne de Tinguy,Science politique,summary,L’Ukraine après Maïdan,L’Ukraine après Maïdan:Vers de nouveaux équili...,Anne de Tinguy,NaN,...,NaN,228,233,10.3917/ifri.demon.2014.01.0228,NaN,NaN,"La révolte déclenchée par la décision, le 21 n...",Chapitre d’ouvrage,Kourkov,Andreï Kourkov
88958,88958,https://shs.cairn.info/magazine-books-2011-9-p...,Poutine en héros littéraire,NaN,NaN,summary,Poutine en héros littéraire,Poutine en héros littéraire,NaN,Books,...,9,40,43,NaN,NaN,NaN,"R êvé, réel ou transfiguré, son personnage app...",Article de magazine,Kourkov,Andreï Kourkov
88959,88959,https://shs.cairn.info/revue-raison-presente-2...,Au risque des arts contemporains,Christian Ruby,NaN,summary,Au risque des arts contemporains,Au risque des arts contemporains,Christian Ruby,Raison présente,...,2,117,119,10.3917/rpre.222.0117,https://shs.cairn.info/article/RPRE_222_0117/p...,NaN,"Compte tenu de l’état des affaires du monde, n...",Compte rendu,Kourkov,Andreï Kourkov
88960,88960,https://shs.cairn.info/revue-d-etudes-comparat...,"Korine Amacher, Wladimir Berelowitch (eds), Hi...",Alexandre Sumpf,Histoire,summary,"Korine Amacher, Wladimir Berelowitch (eds), Hi...","Korine Amacher, Wladimir Berelowitch (eds), Hi...",Alexandre Sumpf,Revue d’études comparatives Est-Ouest,...,2,241,244,10.4074/S0338059914002083,https://shs.cairn.info/article/RECEO_452_0241/...,NaN,A lors que l’Histoire – ou plutôt un passé myt...,Article de revue,Kourkov,Andreï Kourkov


### 3. Starting the cleaning

### 3.a First Step : Cleaning the exacts duplicates 

In [11]:
# Remove perfectly identical rows (across all columns)
df = df.drop_duplicates()

# Remove rows where both "resume" and "texte_article" columns are empty or NaN
if "resume" in df.columns and "texte_article" in df.columns:
    df = df[~(df["resume"].isna() & df["texte_article"].isna())]

# Remove completely empty columns
df = df.dropna(axis=1, how='all')

# Remove duplicates excluding the "variante_recherchee" column
columns_without_variante = [col for col in df.columns if col != "variante_recherchee"]
df = df.drop_duplicates(subset=columns_without_variante)

# Remove rows where the year is before 1945
if "annee" in df.columns:
    df["annee"] = pd.to_numeric(df["annee"], errors="coerce")
    df = df[df["annee"] >= 1945]

# Remove "twitter_title", "citation_titre", "citation_auteur" columns if they exist
columns_to_delete = ["twitter_title", "citation_titre", "citation_auteur"]
existing_columns = [col for col in columns_to_delete if col in df.columns]
df = df.drop(columns=existing_columns)

# Reset index
df = df.reset_index(drop=True)

In [12]:
display(df) #72611 rows 

,Unnamed: 0,url,titre,auteur,section,twitter_card,journal,annee,numero,page_debut,page_fin,doi,pdf_url,texte_article,type_document,variante_recherchee,name
0,0,https://shs.cairn.info/orients-occidents-25-si...,Chapitre 11. « Grattez le Russe et vous trouve...,Thierry Camous,Histoire,summary,Hors collection,2007.0,NaN,245,284,NaN,NaN,"C et aphorisme français, remontant sans doute ...",Chapitre d’ouvrage,Catherine 2 de Russie,Catherine II de Russie
1,1,https://shs.cairn.info/revoltes-et-revolutions...,Chapitre 14. La Russie face au servage et à la...,Robert Calvet,NaN,summary,Collection U,2004.0,NaN,206,216,NaN,NaN,L e sujet proposé au concours s’ouvre avec une...,Chapitre d’ouvrage,Catherine 2 de Russie,Catherine II de Russie
2,4,https://shs.cairn.info/histoire-de-la-russie-e...,11 - La technique du pouvoir,Michel Heller,Science politique,summary,Tempus,2015.0,NaN,810,821,NaN,NaN,C atherine II est nettement plus cultivée que ...,Chapitre d’ouvrage,Catherine 2 de Russie,Catherine II de Russie
3,5,https://shs.cairn.info/magazine-les-grands-dos...,"Catherine II, modernisatrice de la Russie",Anna Moretti,Sociologie,summary,Les Grands Dossiers des Sciences Humaines,2017.0,12,15,15,10.3917/gdsh.049.0015,NaN,"Despote éclairée, la Grande Catherine fut... ...",Article de magazine,Catherine 2 de Russie,Catherine II de Russie
4,6,https://shs.cairn.info/revue-communications-20...,Un formalisme germano-russe : le cas de Viktor...,Michel Espagne,Littérature,summary,Communications,2018.0,2,147,162,10.3917/commu.103.0147,https://shs.cairn.info/article/COMMU_103_0147/...,I l est désormais acquis que le formalisme rus...,Article de revue,Catherine 2 de Russie,Catherine II de Russie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72606,88957,https://shs.cairn.info/le-defi-des-emergents--...,L’Ukraine après Maïdan,Anne de Tinguy,Science politique,summary,NaN,2014.0,NaN,228,233,10.3917/ifri.demon.2014.01.0228,NaN,"La révolte déclenchée par la décision, le 21 n...",Chapitre d’ouvrage,Kourkov,Andreï Kourkov
72607,88958,https://shs.cairn.info/magazine-books-2011-9-p...,Poutine en héros littéraire,NaN,NaN,summary,Books,2011.0,9,40,43,NaN,NaN,"R êvé, réel ou transfiguré, son personnage app...",Article de magazine,Kourkov,Andreï Kourkov
72608,88959,https://shs.cairn.info/revue-raison-presente-2...,Au risque des arts contemporains,Christian Ruby,NaN,summary,Raison présente,2022.0,2,117,119,10.3917/rpre.222.0117,https://shs.cairn.info/article/RPRE_222_0117/p...,"Compte tenu de l’état des affaires du monde, n...",Compte rendu,Kourkov,Andreï Kourkov
72609,88960,https://shs.cairn.info/revue-d-etudes-comparat...,"Korine Amacher, Wladimir Berelowitch (eds), Hi...",Alexandre Sumpf,Histoire,summary,Revue d’études comparatives Est-Ouest,2014.0,2,241,244,10.4074/S0338059914002083,https://shs.cairn.info/article/RECEO_452_0241/...,A lors que l’Histoire – ou plutôt un passé myt...,Article de revue,Kourkov,Andreï Kourkov


### 3.b Deleting all the articles that matched with "Russie" and not "Catherine II de Russie".

In [13]:
# Use the EXACT SAME string as in your search
condition = (df['name'].str.contains('Catherine II de Russie', na=False)) & (df['variante_recherchee'] == 'Russie')

# Count before deletion
count_before = condition.sum()
print(f"Number of rows to delete: {count_before}")

if count_before > 0:
    # Display some rows before deletion
    print("\nRows to be deleted (first 5):")
    display(df[condition][['name', 'variante_recherchee']].head())
    
    # Delete the rows
    df = df[~condition]
    
    # Verify deletion
    count_after = condition.sum()
    print(f"Number of matching rows after deletion: {count_after}")
    
    print(f"✅ Successfully deleted {count_before} rows")
else:
    print("No rows to delete - check if the string is exactly the same as in your search")

Number of rows to delete: 0
No rows to delete - check if the string is exactly the same as in your search


In [14]:
display(df) # 72611 rows

,Unnamed: 0,url,titre,auteur,section,twitter_card,journal,annee,numero,page_debut,page_fin,doi,pdf_url,texte_article,type_document,variante_recherchee,name
0,0,https://shs.cairn.info/orients-occidents-25-si...,Chapitre 11. « Grattez le Russe et vous trouve...,Thierry Camous,Histoire,summary,Hors collection,2007.0,NaN,245,284,NaN,NaN,"C et aphorisme français, remontant sans doute ...",Chapitre d’ouvrage,Catherine 2 de Russie,Catherine II de Russie
1,1,https://shs.cairn.info/revoltes-et-revolutions...,Chapitre 14. La Russie face au servage et à la...,Robert Calvet,NaN,summary,Collection U,2004.0,NaN,206,216,NaN,NaN,L e sujet proposé au concours s’ouvre avec une...,Chapitre d’ouvrage,Catherine 2 de Russie,Catherine II de Russie
2,4,https://shs.cairn.info/histoire-de-la-russie-e...,11 - La technique du pouvoir,Michel Heller,Science politique,summary,Tempus,2015.0,NaN,810,821,NaN,NaN,C atherine II est nettement plus cultivée que ...,Chapitre d’ouvrage,Catherine 2 de Russie,Catherine II de Russie
3,5,https://shs.cairn.info/magazine-les-grands-dos...,"Catherine II, modernisatrice de la Russie",Anna Moretti,Sociologie,summary,Les Grands Dossiers des Sciences Humaines,2017.0,12,15,15,10.3917/gdsh.049.0015,NaN,"Despote éclairée, la Grande Catherine fut... ...",Article de magazine,Catherine 2 de Russie,Catherine II de Russie
4,6,https://shs.cairn.info/revue-communications-20...,Un formalisme germano-russe : le cas de Viktor...,Michel Espagne,Littérature,summary,Communications,2018.0,2,147,162,10.3917/commu.103.0147,https://shs.cairn.info/article/COMMU_103_0147/...,I l est désormais acquis que le formalisme rus...,Article de revue,Catherine 2 de Russie,Catherine II de Russie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72606,88957,https://shs.cairn.info/le-defi-des-emergents--...,L’Ukraine après Maïdan,Anne de Tinguy,Science politique,summary,NaN,2014.0,NaN,228,233,10.3917/ifri.demon.2014.01.0228,NaN,"La révolte déclenchée par la décision, le 21 n...",Chapitre d’ouvrage,Kourkov,Andreï Kourkov
72607,88958,https://shs.cairn.info/magazine-books-2011-9-p...,Poutine en héros littéraire,NaN,NaN,summary,Books,2011.0,9,40,43,NaN,NaN,"R êvé, réel ou transfiguré, son personnage app...",Article de magazine,Kourkov,Andreï Kourkov
72608,88959,https://shs.cairn.info/revue-raison-presente-2...,Au risque des arts contemporains,Christian Ruby,NaN,summary,Raison présente,2022.0,2,117,119,10.3917/rpre.222.0117,https://shs.cairn.info/article/RPRE_222_0117/p...,"Compte tenu de l’état des affaires du monde, n...",Compte rendu,Kourkov,Andreï Kourkov
72609,88960,https://shs.cairn.info/revue-d-etudes-comparat...,"Korine Amacher, Wladimir Berelowitch (eds), Hi...",Alexandre Sumpf,Histoire,summary,Revue d’études comparatives Est-Ouest,2014.0,2,241,244,10.4074/S0338059914002083,https://shs.cairn.info/article/RECEO_452_0241/...,A lors que l’Histoire – ou plutôt un passé myt...,Article de revue,Kourkov,Andreï Kourkov


### 3.c Second Step : Separating between two CSV articles, one for articles with exact match for the complete researched author's name and another for the others

In [15]:
# Function to search for a whole word in text, case insensitive
def whole_word_in_text(word, text):
    pattern = r'\b' + re.escape(word) + r'\b'
    return re.search(pattern, text, re.IGNORECASE) is not None

# Verification function: neither variant nor name should appear as whole words in title, author or texte_article
def variant_or_name_absent(row):
    variant = str(row.get("variante_recherchee", "")).lower()
    name = str(row.get("name", "")).lower()

    title = str(row.get("titre", "")).lower()
    author = str(row.get("auteur", "")).lower()
    text = str(row.get("texte_article", "")).lower()

    variant_present = whole_word_in_text(variant, title) or whole_word_in_text(variant, author) or whole_word_in_text(variant, text)
    name_present = whole_word_in_text(name, title) or whole_word_in_text(name, author) or whole_word_in_text(name, text)

    return not (variant_present or name_present)

# Create a new DataFrame with filtered rows
df_verification = df[df.apply(variant_or_name_absent, axis=1)].copy()

# Display
print(f"⚠️  {len(df_verification)} rows where 'variante_recherchee' and 'name' are absent as whole words in 'titre', 'auteur' or 'texte_article':")
display(df_verification)

# Create a DataFrame with rows where variant or name are present
df_restants = df[~df.apply(variant_or_name_absent, axis=1)].copy()

# Save to a new CSV file
df_restants.to_csv("resultats_avec_nom_ou_variante.csv", index=False)

# Display summary
print(f"✅ {len(df_restants)} rows where 'variante_recherchee' or 'name' are present as whole words:")
display(df_restants)

⚠️  42852 rows where 'variante_recherchee' and 'name' are absent as whole words in 'titre', 'auteur' or 'texte_article':


,Unnamed: 0,url,titre,auteur,section,twitter_card,journal,annee,numero,page_debut,page_fin,doi,pdf_url,texte_article,type_document,variante_recherchee,name
0,0,https://shs.cairn.info/orients-occidents-25-si...,Chapitre 11. « Grattez le Russe et vous trouve...,Thierry Camous,Histoire,summary,Hors collection,2007.0,NaN,245,284,NaN,NaN,"C et aphorisme français, remontant sans doute ...",Chapitre d’ouvrage,Catherine 2 de Russie,Catherine II de Russie
1,1,https://shs.cairn.info/revoltes-et-revolutions...,Chapitre 14. La Russie face au servage et à la...,Robert Calvet,NaN,summary,Collection U,2004.0,NaN,206,216,NaN,NaN,L e sujet proposé au concours s’ouvre avec une...,Chapitre d’ouvrage,Catherine 2 de Russie,Catherine II de Russie
2,4,https://shs.cairn.info/histoire-de-la-russie-e...,11 - La technique du pouvoir,Michel Heller,Science politique,summary,Tempus,2015.0,NaN,810,821,NaN,NaN,C atherine II est nettement plus cultivée que ...,Chapitre d’ouvrage,Catherine 2 de Russie,Catherine II de Russie
4,6,https://shs.cairn.info/revue-communications-20...,Un formalisme germano-russe : le cas de Viktor...,Michel Espagne,Littérature,summary,Communications,2018.0,2,147,162,10.3917/commu.103.0147,https://shs.cairn.info/article/COMMU_103_0147/...,I l est désormais acquis que le formalisme rus...,Article de revue,Catherine 2 de Russie,Catherine II de Russie
5,7,https://shs.cairn.info/les-laicites-dans-le-mo...,"Chapitre III. Despotismes éclairés, révolution...",Jean Baubérot-Vincent,NaN,summary,Que sais-je ?,2024.0,NaN,33,44,NaN,NaN,"L ’effervescence des Lumières, mise en cause d...",Chapitre de Que sais-je ? / Repères,Catherine 2 de Russie,Catherine II de Russie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72594,88944,https://shs.cairn.info/revue-etudes-2024-4-pag...,Recensions,NaN,NaN,summary,Études,2024.0,4,117,144,10.3917/etu.4314.0117,https://shs.cairn.info/article/ETU_4314_0117/p...,"■ Depuis quelques années, l’œuvre de Philippe ...",Compte rendu,Kourkov,Andreï Kourkov
72605,88956,https://shs.cairn.info/le-djihad-de-la-vache--...,"1. La vache, le tribut et la tribu : l’ethnie ...",Giovanni Zanoletti,NaN,summary,Recherches internationales,2023.0,NaN,71,129,NaN,NaN,Tiré d’une étude du vétérinaire colonial Briza...,Chapitre d’ouvrage,Kourkov,Andreï Kourkov
72606,88957,https://shs.cairn.info/le-defi-des-emergents--...,L’Ukraine après Maïdan,Anne de Tinguy,Science politique,summary,NaN,2014.0,NaN,228,233,10.3917/ifri.demon.2014.01.0228,NaN,"La révolte déclenchée par la décision, le 21 n...",Chapitre d’ouvrage,Kourkov,Andreï Kourkov
72608,88959,https://shs.cairn.info/revue-raison-presente-2...,Au risque des arts contemporains,Christian Ruby,NaN,summary,Raison présente,2022.0,2,117,119,10.3917/rpre.222.0117,https://shs.cairn.info/article/RPRE_222_0117/p...,"Compte tenu de l’état des affaires du monde, n...",Compte rendu,Kourkov,Andreï Kourkov


✅ 29759 rows where 'variante_recherchee' or 'name' are present as whole words:


,Unnamed: 0,url,titre,auteur,section,twitter_card,journal,annee,numero,page_debut,page_fin,doi,pdf_url,texte_article,type_document,variante_recherchee,name
3,5,https://shs.cairn.info/magazine-les-grands-dos...,"Catherine II, modernisatrice de la Russie",Anna Moretti,Sociologie,summary,Les Grands Dossiers des Sciences Humaines,2017.0,12,15,15,10.3917/gdsh.049.0015,NaN,"Despote éclairée, la Grande Catherine fut... ...",Article de magazine,Catherine 2 de Russie,Catherine II de Russie
16,18,https://shs.cairn.info/livres-anciens-lectures...,Le long chemin de Bueve de Hantone : le « dram...,Anna Maria Babbi,Littérature,summary,NaN,2010.0,NaN,263,276,10.3917/oj.zink.2010.01.0263,NaN,"L e 26 décembre 1758, au théâtre de San Moisè,...",Chapitre d’ouvrage,Catherine 2 de Russie,Catherine II de Russie
20,23,https://shs.cairn.info/revue-relations-interna...,Qu’est-ce qu’une crise d’Orient ?,Henry Laurens,Histoire,summary,Relations internationales,2017.0,3,9,16,10.3917/ri.171.0009,https://shs.cairn.info/article/RI_171_0009/pdf...,"D ans la seconde moitié du xviii e siècle, l’E...",Article de revue,Catherine 2 de Russie,Catherine II de Russie
21,24,https://shs.cairn.info/revue-regard-libre-2022...,"Le sexe, un objet politique ?",Olivier Meuwly,NaN,summary,Le Regard Libre,2022.0,8,21,23,10.3917/regli.088.0021,https://shs.cairn.info/article/REGLI_088_0021/...,"L’historien français René Rémond, décédé en 20...",Article de revue,Catherine 2 de Russie,Catherine II de Russie
143,155,https://shs.cairn.info/les-revers-de-l-amour-u...,Catherine II de Russie et Grigori Potemkine,Sabine Melchior-Bonnet,Histoire,summary,Hors collection,2019.0,NaN,231,250,NaN,NaN,"C atherine II, impératrice de Russie, a des am...",Chapitre d’ouvrage,Catherine 2 de Russie,Catherine II de Russie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72602,88952,https://shs.cairn.info/dictionnaire-amoureux-d...,C,Tetiana Andrushchuk,NaN,summary,Dictionnaire amoureux,2022.0,NaN,111,133,NaN,NaN,"S ur la place centrale de Lviv, mais aussi aux...",Chapitre d’ouvrage,Kourkov,Andreï Kourkov
72603,88954,https://shs.cairn.info/le-livre-noir-de-vladim...,3. La fuite en avant de Vladimir Poutine vers ...,Galia Ackerman,NaN,summary,NaN,2023.0,NaN,85,106,10.3917/perri.acker.2023.01.0085,NaN,S a réélection en 2012 – et la perspective qui...,Chapitre d’ouvrage,Kourkov,Andreï Kourkov
72604,88955,https://shs.cairn.info/l-ukraine-de-l-independ...,Pour aller plus loin,NaN,NaN,summary,Idées reçues,2023.0,NaN,185,189,NaN,NaN,"L es titres en français sont édités, sauf préc...",Chapitre d’ouvrage,Kourkov,Andreï Kourkov
72607,88958,https://shs.cairn.info/magazine-books-2011-9-p...,Poutine en héros littéraire,NaN,NaN,summary,Books,2011.0,9,40,43,NaN,NaN,"R êvé, réel ou transfiguré, son personnage app...",Article de magazine,Kourkov,Andreï Kourkov


In [18]:
df_restants.to_csv("CAIRN_29759_P1_OK.csv", index=False, encoding='utf-8', sep=';')

### 3.d Third Step : Cleaning the CSV of articles without exact match

### 3.d.1 Separating articles matched based on the first name or last name or neither 

Firstly I had to create by hand a csv with two columns, containing each "variante_recherchee" following the order Name First Name and in another column the version following the First Name order to exchange each of them in the csv, so I know which word is the name or first name for each research, based on the order. 

In [19]:
df_rempl = pd.read_csv("remplacement_variantes.csv")
display(df_rempl.head())

,variante_recherchee,variante_recherchee_2
0,Pouchkine Alexandre,Alexandre Pouchkine
1,Gogol Nicolas,Nicolas Gogol
2,Tolstoï Alexis,Alexis Tolstoi
3,Tourgueniev Ivan,Ivan Tourgueniev
4,Dostoïevski Fiodor,Fiodor Dostoïevski


In [20]:
# Find rows in df_verification that still match a variant from the CSV
matches_restants = df_verification[df_verification['variante_recherchee'].isin(df_rempl['variante_recherchee'])]

print(f"Number of matches: {len(matches_restants)}")

# Create replacement dictionary
remplacement_dict = dict(zip(df_rempl['variante_recherchee'], df_rempl['variante_recherchee_2']))

# Apply replacement
df_verif_mod = df_verification.copy()
df_verif_mod['variante_recherchee'] = df_verif_mod['variante_recherchee'].replace(remplacement_dict)

# Check number of modifications
nombre_modifs = (df_verif_mod['variante_recherchee'] != df_verification['variante_recherchee']).sum()
print(f"Number of modified rows: {nombre_modifs}")

Number of matches: 5580
Number of modified rows: 5580


In [21]:
def word_match(token, text):
    """Check if a token appears as a whole word (case-insensitive)."""
    return bool(re.search(rf"\b{re.escape(token)}\b", text, flags=re.IGNORECASE))

def classify_match_type_exclusive(row):
    """
    EXCLUSIVE classification:
    - "prénom": ONLY first name is present
    - "nom": ONLY last name is present  
    - "both": BOTH are present
    - "none": NEITHER is present
    """
    name = str(row.get("name", "")).strip()
    texte_article = str(row.get("texte_article", ""))

    # Split the full name into tokens (KEEP ORIGINAL CASE)
    tokens = [t for t in name.split() if t]
    if len(tokens) == 0:
        return ("none", "")

    first_name = tokens[0]
    last_name = tokens[-1]

    # Check for both first and last name (remains case-insensitive)
    has_first_name = word_match(first_name, texte_article)
    has_last_name = word_match(last_name, texte_article)

    if has_first_name and has_last_name:
        return ("both", f"{first_name} {last_name}")
    elif has_first_name and not has_last_name:
        return ("prénom", first_name)  # ← ORIGINAL CASE
    elif has_last_name and not has_first_name:
        return ("nom", last_name)      # ← ORIGINAL CASE
    else:
        return ("none", "")

# Apply EXCLUSIVE classification to df_verif_mod
df_verif_mod["match_type"], df_verif_mod["mot_trouve"] = zip(*df_verif_mod.apply(classify_match_type_exclusive, axis=1))

# Separate into four EXCLUSIVE DataFrames
df_prenom_only = df_verif_mod[df_verif_mod["match_type"] == "prénom"].copy()
df_nom_only = df_verif_mod[df_verif_mod["match_type"] == "nom"].copy()
df_both_match = df_verif_mod[df_verif_mod["match_type"] == "both"].copy()
df_no_match = df_verif_mod[df_verif_mod["match_type"] == "none"].copy()

print(f"📊 MATCH RESULTS (EXCLUSIVE CLASSIFICATION):")
print(f"🔹 FIRST NAME ONLY: {len(df_prenom_only)} articles")
print(f"🔹 LAST NAME ONLY: {len(df_nom_only)} articles") 
print(f"🔹 FIRST AND LAST NAME: {len(df_both_match)} articles")
print(f"🔹 NONE: {len(df_no_match)} articles")
print(f"📈 Total: {len(df_verif_mod)} articles")

# Verification
total_categories = len(df_prenom_only) + len(df_nom_only) + len(df_both_match) + len(df_no_match)
print(f"✅ Verification: {total_categories} = {len(df_verif_mod)}")

# Check that Gogol is now in uppercase
if len(df_nom_only) > 0:
    gogol_rows = df_nom_only[df_nom_only['mot_trouve'] == 'Gogol']
    print(f"\n🔍 Check - Number of rows with 'Gogol' (uppercase) in mot_trouve: {len(gogol_rows)}")


📊 MATCH RESULTS (EXCLUSIVE CLASSIFICATION):
🔹 FIRST NAME ONLY: 2668 articles
🔹 LAST NAME ONLY: 12337 articles
🔹 FIRST AND LAST NAME: 9028 articles
🔹 NONE: 18819 articles
📈 Total: 42852 articles
✅ Verification: 42852 = 42852

🔍 Check - Number of rows with 'Gogol' (uppercase) in mot_trouve: 682


Now I will search for other word with a majuscule imediatly before/after as the matched last name because I know that the other word isn't the first name of the author as the article would be classified in "df_both_match". I will also create a new class if the suspect word in majuscule is also the first word of the sentence, in this case I will need to check for myself. 

In [22]:
def analyser_voisins_directs(row):
    """Analyze words immediately before/after the matched name, treating punctuation as separate 'words'."""
    nom_match = row['mot_trouve']
    texte_article = str(row['texte_article'])

    # EXCEPTION: always suspect for "Catherine II de Russie"
    if "Catherine II de Russie" in str(row.get("name", "")):
        return "suspect", "Catherine II de Russie", "", "Catherine II de Russie exception"

    # Split text into sentences
    phrases = re.split(r'[.!?]+', texte_article)
    phrase_trouvee = ""
    for phrase in phrases:
        if re.search(rf'\b{re.escape(nom_match)}\b', phrase, re.IGNORECASE):
            phrase_trouvee = phrase.strip()
            break

    if not phrase_trouvee:
        return "phrase_non_trouvee", "", "", ""

    # Split sentence into words, keeping punctuation separate
    mots_phrase = re.findall(r'\w+|[^\w\s]', phrase_trouvee, re.UNICODE)
    position_nom = -1
    mot_avant_direct = ""
    mot_apres_direct = ""

    # Find the position of the matched name
    for i, mot in enumerate(mots_phrase):
        if re.fullmatch(re.escape(nom_match), mot, re.IGNORECASE):
            position_nom = i
            if i > 0:
                mot_avant_direct = mots_phrase[i - 1]
            if i < len(mots_phrase) - 1:
                mot_apres_direct = mots_phrase[i + 1]
            break

    if position_nom == -1:
        return "nom_non_localise", "", "", ""

    # Only consider capitalization if the adjacent "word" is not punctuation
    punctuation = {",", ";", ":", "(", ")", "—", "-", "–", "."}
    avant_majuscule = mot_avant_direct and mot_avant_direct[0].isupper() and mot_avant_direct not in punctuation
    apres_majuscule = mot_apres_direct and mot_apres_direct[0].isupper() and mot_apres_direct not in punctuation

    # Determine category
    if avant_majuscule and position_nom == 1:
        categorie = "suspect_premier_mot"
    elif avant_majuscule or apres_majuscule:
        categorie = "suspect"
    else:
        categorie = "valide"

    return categorie, mot_avant_direct, mot_apres_direct, phrase_trouvee


# === Apply the analysis to the DataFrame ===
print("🔍 ANALYZING WORDS DIRECTLY BEFORE/AFTER MATCHED NAME")

df_nom_only['categorie_majuscule'], df_nom_only['mot_avant_direct'], df_nom_only['mot_apres_direct'], df_nom_only['phrase_suspecte'] = zip(
    *df_nom_only.apply(analyser_voisins_directs, axis=1)
)

# Separate results by category
df_valide = df_nom_only[df_nom_only['categorie_majuscule'] == 'valide'].copy()
df_suspect = df_nom_only[df_nom_only['categorie_majuscule'] == 'suspect'].copy()
df_suspect_premier_mot = df_nom_only[df_nom_only['categorie_majuscule'] == 'suspect_premier_mot'].copy()
df_autres = df_nom_only[~df_nom_only['categorie_majuscule'].isin(['valide', 'suspect', 'suspect_premier_mot'])].copy()

# Display summary
print(f"\n📊 ANALYSIS RESULTS:")
print(f"✅ Validated names: {len(df_valide)} articles")
print(f"⚠️  Suspect names: {len(df_suspect)} articles")
print(f"🔴 Suspect names - first word: {len(df_suspect_premier_mot)} articles")
print(f"❓ Other cases: {len(df_autres)} articles")

# Save results to CSV
df_valide.to_csv("noms_valides.csv", index=False)
df_suspect.to_csv("noms_suspects.csv", index=False)
df_suspect_premier_mot.to_csv("noms_suspects_premier_mot.csv", index=False)

print(f"\n💾 Files saved:")
print(f"   - noms_valides.csv : {len(df_valide)} articles (with phrase_suspecte column)")
print(f"   - noms_suspects.csv : {len(df_suspect)} articles (with phrase_suspecte column)")
print(f"   - noms_suspects_premier_mot.csv : {len(df_suspect_premier_mot)} articles (with phrase_suspecte column)")



🔍 ANALYZING WORDS DIRECTLY BEFORE/AFTER MATCHED NAME

📊 ANALYSIS RESULTS:
✅ Validated names: 9620 articles
⚠️  Suspect names: 2545 articles
🔴 Suspect names - first word: 172 articles
❓ Other cases: 0 articles

💾 Files saved:
   - noms_valides.csv : 9620 articles (with phrase_suspecte column)
   - noms_suspects.csv : 2545 articles (with phrase_suspecte column)
   - noms_suspects_premier_mot.csv : 172 articles (with phrase_suspecte column)


In [25]:
df_suspect_premier_mot.to_excel("CAIRN_Suspect_premier_mot.xlsx")

I read every article with suspect names that also appear as the first word. The reason is that I won’t keep articles containing a word starting with a capital letter around the name of the Russian author. However, it can sometimes be capitalized simply because it is the first word of the sentence and not because the text refers to someone else, so I will check these exceptions myself.

In [32]:
# Load CSV
df_suspect_premier_mot = pd.read_csv("CAIRN_Suspect_premier_mot.csv", sep=';')

# List of indices to display and delete
indices = [6,7,9,11,12,17,79,81,90,95,96,97,98,102,103,104,120,123,124,125,126,127,128,129,130,131,132,133,134,135,139,140,141,142,143,144,145,146,147,148,151,152,153,155,156,158,159,160,161,162,163,164,165,166,167,168,169,170,171]

# Display corresponding rows
display(df_suspect_premier_mot.iloc[indices].head())

# Delete these rows from DataFrame
df_suspect_premier_mot = df_suspect_premier_mot.drop(indices)

# Save updated CSV
df_suspect_premier_mot.to_csv("NOM_premiers_mots_ok.csv", index=False)


,Unnamed: 0,url,titre,auteur,section,twitter_card,journal,annee,numero,page_debut,...,texte_article,type_document,variante_recherchee,name,match_type,mot_trouve,categorie_majuscule,mot_avant_direct,mot_apres_direct,phrase_suspecte
6,17045,https://shs.cairn.info/tolstoi-la-dechirure--9...,Tolstoï : une vie,Annie Anargyros,Littérature,summary,Champs psychanalytiques,1999.0,NaN,21,...,Léon Tolstoï est issu de la plus vieille noble...,Chapitre d’ouvrage,A. Tolstoi,Alexis Tolstoï,nom,Tolstoï,suspect_premier_mot,Léon,est,Léon Tolstoï est issu de la plus vieille noble...
7,17074,https://shs.cairn.info/pensee-et-langage--9782...,Chapitre 6. Étude du développement des concept...,Lev Semionovitch Vygotski,NaN,summary,NaN,2019.0,NaN,281,...,L a question du développement des concepts sci...,Chapitre d’ouvrage,A. Tolstoi,Alexis Tolstoï,nom,Tolstoï,suspect_premier_mot,Léon,",","Léon Tolstoï, ce très profond connaisseur de l..."
9,17137,https://shs.cairn.info/revue-de-l-histoire-des...,« La vérité est qu’on ne sait plus que devenir...,Sylvain Milbach,NaN,summary,Revue de l'histoire des religions,2023.0,3.0,485,...,« Notre mal est de nous sentir moins homme qu...,Article de revue,A. Tolstoi,Alexis Tolstoï,nom,Tolstoï,suspect_premier_mot,Léon,connut,Léon Tolstoï connut alors un grand succès
11,17210,https://shs.cairn.info/revue-vingtieme-siecle-...,Pour une approche historique comparée des musi...,Didier Francfort,Histoire,summary,Vingtième Siècle. Revue d'histoire,2005.0,1.0,85,...,L es musiques militaires ont souvent mauvaise ...,Article de revue,A. Tolstoi,Alexis Tolstoï,nom,Tolstoï,suspect_premier_mot,Léon,",","Léon Tolstoï, dans La sonate à Kreutzer, a vou..."
12,17274,https://shs.cairn.info/ordre-moins-le-pouvoir-...,II. Anarchistes & anarchismes,Normand Baillargeon,Philosophie,summary,Éléments,2008.0,NaN,43,...,O n a proposé bien des précurseurs à l’anarchi...,Chapitre d’ouvrage,A. Tolstoi,Alexis Tolstoï,nom,Tolstoï,suspect_premier_mot,Léon,(,Léon Tolstoï (1828-1910) développa en Russie u...


In [34]:
display(df_suspect_premier_mot)

,Unnamed: 0,url,titre,auteur,section,twitter_card,journal,annee,numero,page_debut,...,texte_article,type_document,variante_recherchee,name,match_type,mot_trouve,categorie_majuscule,mot_avant_direct,mot_apres_direct,phrase_suspecte
0,15740,https://shs.cairn.info/la-mort--9782130591429-...,"Chapitre premier - La mort, ses représentation...",Alexandrine Schniewind,Littérature,summary,Que sais-je ?,2014.0,NaN,9,...,L a conscience de la mort n’a pas toujours exi...,Chapitre de Que sais-je ? / Repères,N. Gogol,Nicolas Gogol,nom,Gogol,suspect_premier_mot,Nikolaï,(,"Nikolaï Gogol (1809-1852), quant à lui, formul..."
1,16012,https://shs.cairn.info/journal-mondes1-2013-2-...,The Pahlen Commission and the Re-Establishment...,Alexander Morrison,History,summary,Monde(s),2013.0,2.0,45,...,The “Imperial Visitor” summons up a faintly ri...,Journal article,N. Gogol,Nicolas Gogol,nom,Gogol,suspect_premier_mot,As,’,"As Gogol’s famous play suggests, while the Rev..."
2,16119,https://shs.cairn.info/le-carrosse-litteraire-...,Le hasard escamoté : nature et intransitivité ...,Carsten Meiner,Littérature,summary,Les littéraires,2008.0,NaN,179,...,E ntre les structures socio-historiques déterm...,Chapitre d’ouvrage,N. Gogol,Nicolas Gogol,nom,Gogol,suspect_premier_mot,Chez,un,Chez Gogol un même projet d’idéalisation ne tr...
3,16507,https://shs.cairn.info/la-seduction-de-l-etran...,Index des noms et des matières,NaN,NaN,summary,Quadrige,1987.0,NaN,293,...,A ARNE. ADAM. Adhésion affective : — et croy...,Chapitre d’ouvrage,N. Gogol,Nicolas Gogol,nom,Gogol,suspect_premier_mot,GŒTHE,NaN,GŒTHE GOGOL
4,17011,https://droit.cairn.info/revue-droit-et-litter...,Au cœur des contradictions de Tolstoï : la pro...,Raymond Legeais,Littérature,summary,Revue Droit & Littérature,2022.0,1.0,189,...,Si Tolstoï a pris place parmi les grands écriv...,Article de revue,A. Tolstoi,Alexis Tolstoï,nom,Tolstoï,suspect_premier_mot,Si,a,Si Tolstoï a pris place parmi les grands écriv...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,74693,https://stm.cairn.info/revue-l-information-psy...,Réhabilitation psychosociale et syndrome d'Asp...,Diletta Viezzoli,Psychologie,summary,L'information psychiatrique,2013.0,5.0,385,...,L e syndrome d’Asperger (SA) fait partie des t...,Article de revue,V. Grossman,Vassili Grossman,nom,Grossman,suspect_premier_mot,Selon,et,Selon Grossman et al
149,79915,https://shs.cairn.info/revue-naqd-2002-1-page-45,Ordre et désordre dans les périphéries urbaines,J.-J. Deluz,NaN,summary,NAQD,2002.0,1.0,45,...,"M ême si, dans le domaine de l’urbanisme, les ...",Article de revue,Alexandre Zinoviev,Alexandre Zinoviev,nom,Zinoviev,suspect_premier_mot,Pour,[,"Pour Zinoviev [2], la bureaucratie dirigeante ..."
150,80139,https://shs.cairn.info/revue-naqd-2002-1-page-45,Ordre et désordre dans les périphéries urbaines,J.-J. Deluz,NaN,summary,NAQD,2002.0,1.0,45,...,"M ême si, dans le domaine de l’urbanisme, les ...",Article de revue,Alexandre Zinoviev,Alexandre Zinoviev,nom,Zinoviev,suspect_premier_mot,Pour,[,"Pour Zinoviev [2], la bureaucratie dirigeante ..."
154,82223,https://stm.cairn.info/neurologie-et-orthophon...,Outils de dépistage et d’évaluation clinique d...,Claire Sainson,NaN,summary,NaN,2023.0,NaN,528,...,Kathleen L. DePippo ; Marlene A. Holas & Micha...,Chapitre d’ouvrage,J. Brodsky,Joseph Brodsky,nom,Brodsky,suspect_premier_mot,Reding,",","Reding Brodsky, M"


### 3.d.2 Now I'm will execute the same steps with articles that matched with first and last name. 

In [35]:
def analyze_last_name_only(row):
    """Analyze only the last name to detect direct neighbors."""
    nom_match = row['mot_trouve']  # in df_both_match, mot_trouve contains full name, we take only the last word
    texte_article = str(row['texte_article'])

    # Take the last word (last name)
    nom_tokens = nom_match.split()
    if len(nom_tokens) == 0:
        return "nom_non_localise", "", "", ""
    nom = nom_tokens[-1]

    # EXCEPTION: always suspect for "Catherine II de Russie"
    if "Catherine II de Russie" in str(row.get("name", "")):
        return "suspect", "Catherine II de Russie", "", "Catherine II de Russie exception"

    # Split text into sentences
    phrases = re.split(r'[.!?]+', texte_article)
    phrase_trouvee = ""
    for phrase in phrases:
        if re.search(rf'\b{re.escape(nom)}\b', phrase, re.IGNORECASE):
            phrase_trouvee = phrase.strip()
            break

    if not phrase_trouvee:
        return "phrase_non_trouvee", "", "", ""

    # Split sentence into words, keeping punctuation separate
    mots_phrase = re.findall(r'\w+|[^\w\s]', phrase_trouvee, re.UNICODE)
    position_nom = -1
    mot_avant_direct = ""
    mot_apres_direct = ""

    # Find the position of the matched name
    for i, mot in enumerate(mots_phrase):
        if re.fullmatch(re.escape(nom), mot, re.IGNORECASE):
            position_nom = i
            if i > 0:
                mot_avant_direct = mots_phrase[i - 1]
            if i < len(mots_phrase) - 1:
                mot_apres_direct = mots_phrase[i + 1]
            break

    if position_nom == -1:
        return "nom_non_localise", "", "", ""

    # Only consider capitalization if the adjacent "word" is not punctuation
    punctuation = {",", ";", ":", "(", ")", "—", "-", "–", "."}
    avant_majuscule = mot_avant_direct and mot_avant_direct[0].isupper() and mot_avant_direct not in punctuation
    apres_majuscule = mot_apres_direct and mot_apres_direct[0].isupper() and mot_apres_direct not in punctuation

    # Determine category
    if avant_majuscule and position_nom == 1:
        categorie = "suspect_premier_mot"
    elif avant_majuscule or apres_majuscule:
        categorie = "suspect"
    else:
        categorie = "valide"

    return categorie, mot_avant_direct, mot_apres_direct, phrase_trouvee


# === Apply the analysis to df_both_match (only using the last word / last name) ===
print("🔍 ANALYZING LAST NAME ONLY IN BOTH-MATCH CATEGORY")

df_both_match['categorie_majuscule'], df_both_match['mot_avant_direct'], df_both_match['mot_apres_direct'], df_both_match['phrase_suspecte'] = zip(
    *df_both_match.apply(analyze_last_name_only, axis=1)
)

# Separate results by category
df_valide_both = df_both_match[df_both_match['categorie_majuscule'] == 'valide'].copy()
df_suspect_both = df_both_match[df_both_match['categorie_majuscule'] == 'suspect'].copy()
df_suspect_premier_mot_both = df_both_match[df_both_match['categorie_majuscule'] == 'suspect_premier_mot'].copy()
df_autres_both = df_both_match[~df_both_match['categorie_majuscule'].isin(['valide', 'suspect', 'suspect_premier_mot'])].copy()

# Display summary
print(f"\n📊 ANALYSIS RESULTS (BOTH MATCH - LAST NAME ONLY):")
print(f"✅ Validated names: {len(df_valide_both)} articles")
print(f"⚠️  Suspect names: {len(df_suspect_both)} articles")
print(f"🔴 Suspect names - first word: {len(df_suspect_premier_mot_both)} articles")
print(f"❓ Other cases: {len(df_autres_both)} articles")

# Save results to CSV
df_valide_both.to_csv("noms_valides_both.csv", index=False)
df_suspect_both.to_csv("noms_suspects_both.csv", index=False)
df_suspect_premier_mot_both.to_csv("noms_suspects_premier_mot_both.csv", index=False)

print(f"\n💾 Files saved:")
print(f"   - noms_valides_both.csv : {len(df_valide_both)} articles")
print(f"   - noms_suspects_both.csv : {len(df_suspect_both)} articles")
print(f"   - noms_suspects_premier_mot_both.csv : {len(df_suspect_premier_mot_both)} articles")

🔍 ANALYZING LAST NAME ONLY IN BOTH-MATCH CATEGORY

📊 ANALYSIS RESULTS (BOTH MATCH - LAST NAME ONLY):
✅ Validated names: 5583 articles
⚠️  Suspect names: 3339 articles
🔴 Suspect names - first word: 106 articles
❓ Other cases: 0 articles

💾 Files saved:
   - noms_valides_both.csv : 5583 articles
   - noms_suspects_both.csv : 3339 articles
   - noms_suspects_premier_mot_both.csv : 106 articles


In [36]:
df_suspect_premier_mot_both.to_excel("CAIRN_Suspect_premier_mot_both.xlsx")

In [38]:
# Load CSV
df_suspect_premier_mot_both = pd.read_csv("noms_suspects_premier_mot_both.csv")

# List of indices to display and delete
indices = [23,24,26,27,30,32,33,39,45,55,63,64,65,72,73,74,75,75,77,78,79,90,93,94,95,96,97,98,99,100,101,102,103,104,105]

# Display corresponding rows
display(df_suspect_premier_mot_both.iloc[indices])

# Delete these rows from DataFrame
df_suspect_premier_mot_both = df_suspect_premier_mot_both.drop(indices)

# Save updated CSV
df_suspect_premier_mot_both.to_csv("NOM_premiers_mots_both_ok.csv", index=False)

,Unnamed: 0,url,titre,auteur,section,twitter_card,journal,annee,numero,page_debut,...,texte_article,type_document,variante_recherchee,name,match_type,mot_trouve,categorie_majuscule,mot_avant_direct,mot_apres_direct,phrase_suspecte
23,17315,https://shs.cairn.info/histoire-de-la-russie-e...,12 - Sur les traces du père,Michel Heller,Histoire,summary,Tempus,2015.0,NaN,1253,...,"L a Khodynka devient un nom funeste, le signe ...",Chapitre d’ouvrage,A. Tolstoi,Alexis Tolstoï,both,Alexis Tolstoï,suspect_premier_mot,Léon,se,Léon Tolstoï se rend lui-même à Samara pour y ...
24,17626,https://shs.cairn.info/pierre-le-grand--978226...,30. De la propagande à l’histoire,Thierry Sarmant,Histoire,summary,Biographies,2020.0,NaN,381,...,T andis que ses successeurs se déchirent tout ...,Chapitre d’ouvrage,Alexis Tolstoï,Alexis Tolstoï,both,Alexis Tolstoï,suspect_premier_mot,Léon,",","Léon Tolstoï, dont la pensée évolua dans une d..."
26,17699,https://shs.cairn.info/histoire-de-la-russie-e...,12 - Sur les traces du père,Michel Heller,Histoire,summary,Tempus,2015.0,NaN,1253,...,"L a Khodynka devient un nom funeste, le signe ...",Chapitre d’ouvrage,Alexis Tolstoï,Alexis Tolstoï,both,Alexis Tolstoï,suspect_premier_mot,Léon,se,Léon Tolstoï se rend lui-même à Samara pour y ...
27,17752,https://shs.cairn.info/2000-ans-de-chretientes...,Une histoire des chrétientés,Gérard Chaliand,Histoire,summary,Hors collection,2000.0,NaN,18,...,"J ésus, en hébreu Yeshua, signifie littéraleme...",Chapitre d’ouvrage,Alexis Tolstoï,Alexis Tolstoï,both,Alexis Tolstoï,suspect_premier_mot,Léon,(,Léon Tolstoï (1828-1910) propose un christiani...
30,18094,https://shs.cairn.info/pierre-le-grand--978226...,30. De la propagande à l’histoire,Thierry Sarmant,Histoire,summary,Biographies,2020.0,NaN,381,...,T andis que ses successeurs se déchirent tout ...,Chapitre d’ouvrage,Alexis Tolstoi,Alexis Tolstoï,both,Alexis Tolstoï,suspect_premier_mot,Léon,",","Léon Tolstoï, dont la pensée évolua dans une d..."
32,18167,https://shs.cairn.info/histoire-de-la-russie-e...,12 - Sur les traces du père,Michel Heller,Histoire,summary,Tempus,2015.0,NaN,1253,...,"L a Khodynka devient un nom funeste, le signe ...",Chapitre d’ouvrage,Alexis Tolstoi,Alexis Tolstoï,both,Alexis Tolstoï,suspect_premier_mot,Léon,se,Léon Tolstoï se rend lui-même à Samara pour y ...
33,18220,https://shs.cairn.info/2000-ans-de-chretientes...,Une histoire des chrétientés,Gérard Chaliand,Histoire,summary,Hors collection,2000.0,NaN,18,...,"J ésus, en hébreu Yeshua, signifie littéraleme...",Chapitre d’ouvrage,Alexis Tolstoi,Alexis Tolstoï,both,Alexis Tolstoï,suspect_premier_mot,Léon,(,Léon Tolstoï (1828-1910) propose un christiani...
39,29433,https://shs.cairn.info/revue-les-temps-moderne...,Les lieux de Bruno Schulz : Drogobytch et Trus...,Marc Sagnol,Sociologie,summary,Les Temps Modernes,2005.0,1.0,151,...,D éjà les multiples coupoles des églises et de...,Article de revue,Léon Tolstoi,Léon Tolstoï,both,Léon Tolstoï,suspect_premier_mot,Alexis,",","Alexis Tolstoï, correspondant militaire pendan..."
45,30939,https://shs.cairn.info/revue-les-temps-moderne...,Les lieux de Bruno Schulz : Drogobytch et Trus...,Marc Sagnol,Sociologie,summary,Les Temps Modernes,2005.0,1.0,151,...,D éjà les multiples coupoles des églises et de...,Article de revue,Léon Tolstoï,Léon Tolstoï,both,Léon Tolstoï,suspect_premier_mot,Alexis,",","Alexis Tolstoï, correspondant militaire pendan..."
55,34595,https://shs.cairn.info/revue-les-temps-moderne...,Les lieux de Bruno Schulz : Drogobytch et Trus...,Marc Sagnol,Sociologie,summary,Les Temps Modernes,2005.0,1.0,151,...,D éjà les multiples coupoles des églises et de...,Article de revue,Leon Tolstoï,Léon Tolstoï,both,Léon Tolstoï,suspect_premier_mot,Alexis,",","Alexis Tolstoï, correspondant militaire pendan..."


### 4. Aggregating all the csv I have validated to form the final CAIRN csv.

In [40]:
dif=pd.read_csv("CAIRN_29759_P1_OK.csv",sep=';')
dif2=pd.read_csv("noms_valides.csv")
dif3=pd.read_csv("NOM_premiers_mots_ok.csv")
dif4=pd.read_csv("noms_valides_both.csv")
dif5=pd.read_csv("NOM_premiers_mots_both_ok.csv")

In [41]:
# Concatenate the CSVs
pd.concat([dif, dif2, dif3, dif4, dif5], ignore_index=True).to_csv("CAIRN_TOUT_OK.csv", index=False)

# Read the concatenated CSV
dif6 = pd.read_csv("CAIRN_TOUT_OK.csv")

# Keep only columns present in df
dif6 = dif6[df.columns.intersection(dif6.columns)]

# Display result
display(dif6) # 45147 rows

,Unnamed: 0,url,titre,auteur,section,twitter_card,journal,annee,numero,page_debut,page_fin,doi,pdf_url,texte_article,type_document,variante_recherchee,name
0,5,https://shs.cairn.info/magazine-les-grands-dos...,"Catherine II, modernisatrice de la Russie",Anna Moretti,Sociologie,summary,Les Grands Dossiers des Sciences Humaines,2017.0,12,15,15,10.3917/gdsh.049.0015,NaN,"Despote éclairée, la Grande Catherine fut... ...",Article de magazine,Catherine 2 de Russie,Catherine II de Russie
1,18,https://shs.cairn.info/livres-anciens-lectures...,Le long chemin de Bueve de Hantone : le « dram...,Anna Maria Babbi,Littérature,summary,NaN,2010.0,NaN,263,276,10.3917/oj.zink.2010.01.0263,NaN,"L e 26 décembre 1758, au théâtre de San Moisè,...",Chapitre d’ouvrage,Catherine 2 de Russie,Catherine II de Russie
2,23,https://shs.cairn.info/revue-relations-interna...,Qu’est-ce qu’une crise d’Orient ?,Henry Laurens,Histoire,summary,Relations internationales,2017.0,3,9,16,10.3917/ri.171.0009,https://shs.cairn.info/article/RI_171_0009/pdf...,"D ans la seconde moitié du xviii e siècle, l’E...",Article de revue,Catherine 2 de Russie,Catherine II de Russie
3,24,https://shs.cairn.info/revue-regard-libre-2022...,"Le sexe, un objet politique ?",Olivier Meuwly,NaN,summary,Le Regard Libre,2022.0,8,21,23,10.3917/regli.088.0021,https://shs.cairn.info/article/REGLI_088_0021/...,"L’historien français René Rémond, décédé en 20...",Article de revue,Catherine 2 de Russie,Catherine II de Russie
4,155,https://shs.cairn.info/les-revers-de-l-amour-u...,Catherine II de Russie et Grigori Potemkine,Sabine Melchior-Bonnet,Histoire,summary,Hors collection,2019.0,NaN,231,250,NaN,NaN,"C atherine II, impératrice de Russie, a des am...",Chapitre d’ouvrage,Catherine 2 de Russie,Catherine II de Russie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45142,63389,https://shs.cairn.info/revue-des-sciences-phil...,Recension des revues,NaN,NaN,summary,Revue des sciences philosophiques et théologiques,2009.0,4.0,891,933,10.3917/rspt.934.0891,https://shs.cairn.info/article/RSPT_934_0891/p...,A NALECTA BOLLANDIANA. 125 (2007) 2 — Gascou J...,Article de revue,Isaac Babel,Isaac Babel
45143,63554,https://shs.cairn.info/les-decouvreurs--978222...,Quinzième partie. L’étude du présent,Daniel Boorstin,NaN,summary,Bouquins / la Collection,2022.0,NaN,627,692,NaN,NaN,"E n 1537, le grand cartographe portugais, Pedr...",Chapitre d’ouvrage,Isaac Babel,Isaac Babel
45144,66178,https://shs.cairn.info/revue-litterature-2012-...,Beckett et la langue des maîtres,Anthony Cordingley,Sciences de l’éducation,summary,Littérature,2012.0,3.0,90,103,10.3917/litt.167.0090,https://shs.cairn.info/article/LITT_167_0090/p...,N abokov estime que le français de Beckett est...,Article de revue,Vladimir Nabokov,Vladimir Nabokov
45145,66731,https://shs.cairn.info/les-russes-blancs--9782...,La désintégration,Alexandre Jevakhoff,Histoire,summary,Texto,2011.0,NaN,55,66,NaN,NaN,"À Petrograd, la révolution fait une victime di...",Chapitre d’ouvrage,Vladimir Nabokov,Vladimir Nabokov


### 4.a Last clean up 

This code first removes exact duplicate rows, then merges partial duplicates (rows with the same URL and article text) by concatenating different values from other columns with " | " separators.

In [42]:
# Remove exact duplicates (identical rows across all columns)
df_unique = dif6.drop_duplicates()

# Function to concatenate unique values of a column within a group
def concat_unique_values(series):
    uniques = series.dropna().unique()
    if len(uniques) == 1:
        return uniques[0]
    else:
        return " | ".join(str(u) for u in uniques)

# Group by partial duplicates on ('url', 'texte_article')
df_merged = (
    df_unique
    .groupby(['url', 'texte_article'])  # <-- double key: URL + texte_article
    .agg(concat_unique_values)
    .reset_index()
)

# Display result
display(df_merged.head()) # 16401 rows

# Save to new CSV (optional)
df_merged.to_csv("CAIRN_real_final.csv", index=False)


,url,texte_article,Unnamed: 0,titre,auteur,section,twitter_card,journal,annee,numero,page_debut,page_fin,doi,pdf_url,type_document,variante_recherchee,name
0,https://droit.cairn.info/Economie-des-donnees-...,L 'objectif de ce chapitre est d'envisager à q...,86147,IV. Les comportements en matière de vie privée...,Fabrice Rochelandet,Sociologie,summary,Repères,2010.0,,67,87,,,Chapitre de Que sais-je ? / Repères,Ackerman,Galia Ackerman
1,https://droit.cairn.info/agressions-sexuelles-...,"L a révélation des faits, leur dénonciation au...",66457,Chapitre 2. Le combat judiciaire,Gilles Antonowicz,Travail social,summary,Hors collection,2002.0,,61,140,,,Chapitre d’ouvrage,Vladimir Nabokov,Vladimir Nabokov
2,https://droit.cairn.info/argumentation-et-droi...,L ’œuvre de Perelman repose sur une double cri...,88602 | 88470,Chapitre Premier. La critique du positivisme e...,Guillaume Vannier,,summary,L'Interrogation philosophique,2001.0,,19,56,,,Chapitre d’ouvrage,Sorokine | V. Sorokine,Vladimir Sorokine
3,https://droit.cairn.info/au-service-du-droit-i...,M e Léopold Dor (1881-1960) a été qualifié de ...,51634,Regard historien sur la Branche française (192...,Dzovinar Kévonian,,summary,,2023.0,,347,358,10.3917/epas.kesse.2023.01.0347,,Chapitre d’ouvrage,Mandelstam,Ossip Mandelstam
4,https://droit.cairn.info/condamner--9782749246...,L e syntagme « justice pénale » doit être ente...,72147,1. Domination versus régulation. La justice pé...,Dan Kaminski,Gestion,summary,Trajets,2015.0,,33,101,,,Chapitre d’ouvrage,Grossman,Vassili Grossman


### 4.b Final verification

In [46]:
# --- Extract last name (family name = last token) ---
def extract_last_name(full_name):
    if pd.isna(full_name):
        return None
    full_name = re.sub(r'\s+', ' ', full_name.strip())
    return full_name.split(' ')[-1].lower()

# --- Check if author last name is absent from the text ---
def author_name_absent(row):
    text = str(row['texte_article']).lower()

    possible_names = set()

    name_variant = extract_last_name(row['variante_recherchee'])
    canonical_name = extract_last_name(row['name'])

    if name_variant:
        possible_names.add(name_variant)
    if canonical_name:
        possible_names.add(canonical_name)

    # True if NONE of the possible last names are found in the text
    return not any(name in text for name in possible_names)

# --- Apply to df_merged ---
df_merged['author_name_absent'] = df_merged.apply(author_name_absent, axis=1)

# Rows where the author name was NOT found
df_name_not_found = df_merged[df_merged['author_name_absent']]

print(f"Rows where author name is NOT found: {len(df_name_not_found)}")
display(df_name_not_found[['variante_recherchee', 'name', 'texte_article']])


Rows where author name is NOT found: 170


,variante_recherchee,name,texte_article
60,Grossman,Vassili Grossman,E : Davignon Report(s) D : Davignon Bericht(...
62,Grossman,Vassili Grossman,E : Intra-Community Trade D : Innergemeinsch...
63,Grossman,Vassili Grossman,L e lecteur trouvera La numérotation des arti...
64,Grossman,Vassili Grossman,E : Federalism D : Föderalismus Le débat en...
65,Grossman,Vassili Grossman,"L e gatt, remplacé en 1995 par l’Organisation ..."
...,...,...,...
15522,Grossman,Vassili Grossman,L ’idée du Neutre chez Barthes n’est pas seule...
15740,Pasternak Boris | Boris Pasternak | Ilya Ehren...,Boris Pasternak | Ilya Ehrenbourg | Evguénia G...,L ’interprétation de l’histoire du régime comm...
16115,Grossman,Vassili Grossman,V an Gogh le suicidé de la société parut chez ...
16180,Grossman,Vassili Grossman,A quantitative perspective on European institu...


In [47]:
# Keep only rows where the author name WAS found
df_merged_clean = df_merged[~df_merged['author_name_absent']].copy()

print(f"Rows kept after removing missing-author cases: {len(df_merged_clean)}")
display(df_merged_clean.head())

df_merged_clean.to_csv("CAIRN_final_cleaned.csv", index=False)

Rows kept after removing missing-author cases: 16231


,url,texte_article,Unnamed: 0,titre,auteur,section,twitter_card,journal,annee,numero,page_debut,page_fin,doi,pdf_url,type_document,variante_recherchee,name,nom_auteur_absent,author_name_absent
0,https://droit.cairn.info/Economie-des-donnees-...,L 'objectif de ce chapitre est d'envisager à q...,86147,IV. Les comportements en matière de vie privée...,Fabrice Rochelandet,Sociologie,summary,Repères,2010.0,,67,87,,,Chapitre de Que sais-je ? / Repères,Ackerman,Galia Ackerman,False,False
1,https://droit.cairn.info/agressions-sexuelles-...,"L a révélation des faits, leur dénonciation au...",66457,Chapitre 2. Le combat judiciaire,Gilles Antonowicz,Travail social,summary,Hors collection,2002.0,,61,140,,,Chapitre d’ouvrage,Vladimir Nabokov,Vladimir Nabokov,False,False
2,https://droit.cairn.info/argumentation-et-droi...,L ’œuvre de Perelman repose sur une double cri...,88602 | 88470,Chapitre Premier. La critique du positivisme e...,Guillaume Vannier,,summary,L'Interrogation philosophique,2001.0,,19,56,,,Chapitre d’ouvrage,Sorokine | V. Sorokine,Vladimir Sorokine,False,False
3,https://droit.cairn.info/au-service-du-droit-i...,M e Léopold Dor (1881-1960) a été qualifié de ...,51634,Regard historien sur la Branche française (192...,Dzovinar Kévonian,,summary,,2023.0,,347,358,10.3917/epas.kesse.2023.01.0347,,Chapitre d’ouvrage,Mandelstam,Ossip Mandelstam,False,False
4,https://droit.cairn.info/condamner--9782749246...,L e syntagme « justice pénale » doit être ente...,72147,1. Domination versus régulation. La justice pé...,Dan Kaminski,Gestion,summary,Trajets,2015.0,,33,101,,,Chapitre d’ouvrage,Grossman,Vassili Grossman,False,False
